# Recommender Systems 4 - Content-based Filtering

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Load data

In [2]:
df = pd.read_csv('data/Seattle_hotels.csv', encoding="latin-1")
df.shape

(152, 3)

### Examine data

In [3]:
pd.set_option('max_colwidth', None)

In [4]:
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive. Unwind in the bar, and enjoy American cuisine for breakfast, lunch and dinner in our restaurant. The 24-hour Pavilion Pantry? stocks a variety of snacks, drinks and sundries."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kitchen & Bar by James Beard award winning chef Thierry Rautureau."
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the award-winning \nCrowne Plaza Hotel Seattle ? Downtown offers an exceptional blend of service, style and comfort. You?ll notice Cool, Comfortable and Unconventional touches that set us apart as soon as you step inside. Marvel at stunning views of the city lights while relaxing in our new Sleep Advantage? Beds. Enjoy complimentary wireless Internet throughout the hotel and amenities to help you relax like our Temple Spa? Sleep Tight Amenity kits featuring lavender spray and lotions to help you rejuvenate and unwind. Enjoy an invigorating workout at our 24-hour fitness center, get dining suggestions from our expert concierge or savor sumptuous cuisine at our Regatta Bar & Grille restaurant where you can enjoy Happy Hour in our lounge daily from 4pm - 7pm and monthly drink specials. Come and experience all that The Emerald City has to offer with us!"
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101","What?s near our hotel downtown Seattle location? The better \nquestion might be what?s not nearby. In addition to being one of the hotels near Pike Place Market, here?s just a small sampling of the rest. Columbia Center, whose Sky View Observatory on the 73rd floor is the tallest public viewing area west of the Mississippi Historic 5th Avenue Theatre, home to musical productions Seattle Central Library, an architectural marvel. Within half a mile: The must-see Pike Place Market, which houses the original Starbucks Pioneer Square, Seattle?s original downtown. Seattle Art Museum. 

### Review token (vocabulary) frequency distribution before removing stop words

In [5]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['desc'], 20)
print('Top 20 words in hotel description before removing stop words')
common_words

Top 20 words in hotel description before removing stop words


[('the', 1245),
 ('and', 1055),
 ('of', 531),
 ('seattle', 493),
 ('to', 471),
 ('in', 443),
 ('our', 355),
 ('hotel', 289),
 ('with', 279),
 ('is', 269),
 ('you', 253),
 ('at', 229),
 ('from', 222),
 ('for', 215),
 ('your', 186),
 ('or', 160),
 ('center', 150),
 ('are', 135),
 ('downtown', 132),
 ('on', 128)]

### Review token (vocabulary) frequency distribution after removing stop words

In [6]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['desc'], 20)
print('Top 20 words in hotel description after removing stop words')
common_words

Top 20 words in hotel description after removing stop words


[('seattle', 493),
 ('hotel', 289),
 ('center', 150),
 ('downtown', 132),
 ('½s', 123),
 ('free', 123),
 ('located', 107),
 ('rooms', 105),
 ('stay', 105),
 ('place', 102),
 ('airport', 99),
 ('market', 97),
 ('space', 94),
 ('enjoy', 93),
 ('pike', 88),
 ('business', 87),
 ('inn', 85),
 ('just', 80),
 ('room', 75),
 ('breakfast', 67)]

### Bigrams frequency distribution before removing stop words

In [7]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['desc'], 20)
print('Top 20 bigrams in hotel description before removing stop words')
common_words

Top 20 bigrams in hotel description before removing stop words


[('in the', 145),
 ('of the', 131),
 ('place market', 85),
 ('pike place', 84),
 ('to the', 80),
 ('from the', 79),
 ('and the', 71),
 ('downtown seattle', 71),
 ('space needle', 65),
 ('at the', 65),
 ('in seattle', 58),
 ('the seattle', 57),
 ('our hotel', 54),
 ('hotel is', 48),
 ('of our', 43),
 ('one of', 41),
 ('the space', 40),
 ('of seattle', 37),
 ('center and', 37),
 ('wi fi', 37)]

### Bigrams frequency distribution after removing stop words

In [8]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['desc'], 20)
print('Top 20 bigrams in hotel description after removing stop words')
common_words

Top 20 bigrams in hotel description after removing stop words


[('place market', 85),
 ('pike place', 84),
 ('downtown seattle', 72),
 ('space needle', 65),
 ('wi fi', 37),
 ('seattleï ½s', 35),
 ('guest rooms', 34),
 ('pacific northwest', 33),
 ('fitness center', 32),
 ('seattle hotel', 32),
 ('lake union', 29),
 ('24 hour', 28),
 ('hotel seattle', 28),
 ('business center', 27),
 ('international airport', 27),
 ('youï ½re', 26),
 ('seattle airport', 26),
 ('high speed', 25),
 ('seattle tacoma', 25),
 ('university washington', 24)]

### Trigrams frequency distribution before removing stop words

In [9]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['desc'], 20)
print('Top 20 trigrams in hotel description before removing stop words')
common_words

Top 20 trigrams in hotel description before removing stop words


[('pike place market', 83),
 ('the space needle', 38),
 ('the heart of', 33),
 ('in the heart', 28),
 ('located in the', 25),
 ('the pacific northwest', 23),
 ('place market and', 23),
 ('university of washington', 23),
 ('one of the', 22),
 ('seattle tacoma international', 21),
 ('tacoma international airport', 21),
 ('easy access to', 20),
 ('free wi fi', 19),
 ('washington state convention', 17),
 ('of downtown seattle', 16),
 ('seattle art museum', 16),
 ('our hotel is', 16),
 ('state convention center', 15),
 ('hotel in seattle', 15),
 ('bed and breakfast', 15)]

### Trigrams frequency distribution after removing stop words

In [10]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['desc'], 20)
print('Top 20 trigrams in hotel description after removing stop words')
common_words

Top 20 trigrams in hotel description after removing stop words


[('pike place market', 83),
 ('seattle tacoma international', 21),
 ('tacoma international airport', 21),
 ('free wi fi', 19),
 ('washington state convention', 17),
 ('seattle art museum', 16),
 ('place market seattle', 16),
 ('state convention center', 15),
 ('high speed internet', 14),
 ('space needle pike', 11),
 ('needle pike place', 11),
 ('south lake union', 11),
 ('downtown seattle hotel', 10),
 ('sea tac airport', 10),
 ('home away home', 9),
 ('heart downtown seattle', 8),
 ('link light rail', 8),
 ('free high speed', 8),
 ('24 hour fitness', 7),
 ('000 square feet', 7)]

### Preprocess data

In [11]:
# Define stop words list
stopwords = nltk.corpus.stopwords.words('english')     # All English Stopwords

In [12]:
# Instantiate Porter stemmer
ps = nltk.PorterStemmer()

In [13]:
# Create function to clean_text
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    text_2 = ' '.join(word for word in text)
    return text_2

In [14]:
# Apply clean_text function to clean hotel description field
df['desc_clean'] = df['desc'].apply(clean_text)
df.head()

,name,address,desc,desc_clean
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive. Unwind in the bar, and enjoy American cuisine for breakfast, lunch and dinner in our restaurant. The 24-hour Pavilion Pantry? stocks a variety of snacks, drinks and sundries.",locat southern tip lake union hilton garden inn seattl downtown hotel perfectli locat busi leisur neighborhood home numer major intern compani includ amazon googl bill melinda gate foundat wealth eclect restaur bar make area seattl one sought local visitor proxim lake union allow visitor take pacif northwest majest sceneri enjoy outdoor activ like kayak sail 2000 sq ft versatil space complimentari busi center stateoftheart av technolog help staff guarante confer cocktail recept wed success refresh sparkl saltwat pool energ latest equip 24hour fit center tast decor flood natur light guest room suit offer everyth need relax stay product unwind bar enjoy american cuisin breakfast lunch dinner restaur 24hour pavilion pantri stock varieti snack drink sundri
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kitchen & Bar by James Beard award winning chef Thierry Rautureau.",locat citi vibrant core sheraton grand seattl provid gateway divers sight sound pacif northwest step front door find gourmet dine bar worldclass shop excit entertain icon local attract includ pike place market space needl chihuli garden glass museum one seven sheraton hotel north america earn esteem grand design guest book confid know theyr receiv highest benchmark product servic offer avail experi recent complet multimilliondollar transform featur new guest room expand sheraton club loung modern meet event space gather stylish new lobbi enjoy privat art collect featur local artist enjoy favorit beverag starbuck sheraton grand featur sever dine option includ loulay kitchen bar jame beard award win chef thierri rautureau
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the award

In [15]:
# Create function to lookup and print hotel description by index
def print_description(index):
    example = df[df.index == index][['desc_clean', 'name']].values[0]
    if len(example) > 0:
        print('Name:', example[1])
        print('Description:', example[0])

In [16]:
print_description(10)

Name: W Seattle
Description: soak vibrant scene live room bar get mix live music dj seri head memor dinner trace offer inspir season fare awardwin atmospher nottobemiss culinari experi downtown seattl work next morn fitï ½ stateoftheart fit center wander explor mani area nearbi attract includ pike place market pioneer squar seattl art museum alway weve got cover time w seattl signatur whateverwheneverï ½ servic wish truli command


In [17]:
print_description(100)

Name: The Baroness Hotel
Description: budget seattl look someth differ histor charm home away home atmospher baro sure make feel like one famili conveni locat first hill proud part virginia mason hospit campu minut harborview medic center swedish hospit baro hotel great option short long term medic patient famili stay whether visit area worldclass medic facil budget vacat goal ensur wonder stay guest amen complimentari internet access two twin one two queen studio mini fridg microwav two twin one queen suit full kitchen laundri facil avail flat screen cabl televis hbo complimentari local call ice vend machin locat lobbi coffe maker hairdryer guestroom room servic avail seven day week rhododendron cafe limit wheelchair access guest librari busi center print fax servic avail 100 nonsmok pet free room air condit fan avail selfpark avail virginia mason hospit fee


In [18]:
df.set_index('name', inplace = True)
df.head()

,address,desc,desc_clean
name,,,
Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the Hilton Garden Inn Seattle Downtown hotel is perfectly located for business and leisure. \nThe neighborhood is home to numerous major international companies including Amazon, Google and the Bill & Melinda Gates Foundation. A wealth of eclectic restaurants and bars make this area of Seattle one of the most sought out by locals and visitors. Our proximity to Lake Union allows visitors to take in some of the Pacific Northwest's majestic scenery and enjoy outdoor activities like kayaking and sailing. over 2,000 sq. ft. of versatile space and a complimentary business center. State-of-the-art A/V technology and our helpful staff will guarantee your conference, cocktail reception or wedding is a success. Refresh in the sparkling saltwater pool, or energize with the latest equipment in the 24-hour fitness center. Tastefully decorated and flooded with natural light, our guest rooms and suites offer everything you need to relax and stay productive. Unwind in the bar, and enjoy American cuisine for breakfast, lunch and dinner in our restaurant. The 24-hour Pavilion Pantry? stocks a variety of snacks, drinks and sundries.",locat southern tip lake union hilton garden inn seattl downtown hotel perfectli locat busi leisur neighborhood home numer major intern compani includ amazon googl bill melinda gate foundat wealth eclect restaur bar make area seattl one sought local visitor proxim lake union allow visitor take pacif northwest majest sceneri enjoy outdoor activ like kayak sail 2000 sq ft versatil space complimentari busi center stateoftheart av technolog help staff guarante confer cocktail recept wed success refresh sparkl saltwat pool energ latest equip 24hour fit center tast decor flood natur light guest room suit offer everyth need relax stay product unwind bar enjoy american cuisin breakfast lunch dinner restaur 24hour pavilion pantri stock varieti snack drink sundri
Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kitchen & Bar by James Beard award winning chef Thierry Rautureau.",locat citi vibrant core sheraton grand seattl provid gateway divers sight sound pacif northwest step front door find gourmet dine bar worldclass shop excit entertain icon local attract includ pike place market space needl chihuli garden glass museum one seven sheraton hotel north america earn esteem grand design guest book confid know theyr receiv highest benchmark product servic offer avail experi recent complet multimilliondollar transform featur new guest room expand sheraton club loung modern meet event space gather stylish new lobbi enjoy privat art collect featur local artist enjoy favorit beverag starbuck sheraton grand featur sever dine option includ loulay kitchen bar jame beard award win chef thierri rautureau
Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the award-wi

### Run TF-IDF vectorizer

In [19]:
# Instantiate TF-IDF vectorizer
#  Notice the ngram_range
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')

In [20]:
# Create a hotel descriptions matrix for each ngram and its TF-IDF score with regard to each hotel description
tfidf_matrix = tf.fit_transform(df['desc_clean'])
tfidf_matrix

<152x26463 sparse matrix of type '<class 'numpy.float64'>'
	with 40087 stored elements in Compressed Sparse Row format>

### Generate similarities matrix (on hotel description)

In [21]:
# cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_similarities = cosine_similarity(tfidf_matrix)
type(cosine_similarities)

numpy.ndarray

In [22]:
cosine_similarities.shape

(152, 152)

In [23]:
hotel_indices = pd.Series(df.index)

In [24]:
hotel_indices

0               Hilton Garden Seattle Downtown
1                       Sheraton Grand Seattle
2                Crowne Plaza Seattle Downtown
3                Kimpton Hotel Monaco Seattle 
4                           The Westin Seattle
                        ...                   
147                  The Halcyon Suite Du Jour
148                                Vermont Inn
149                 Stay Alfred on Wall Street
150         Pike's Place Lux Suites by Barsala
151    citizenM Seattle South Lake Union hotel
Name: name, Length: 152, dtype: object

### Create hotel recommender

In [25]:
def recommendations(name, cosine_similarities=cosine_similarities):
    
    recommended_hotels = []
    
    # Get index of the hotel that matches the name
    hotel_index = hotel_indices[hotel_indices == name].index[0]

    # Create a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[hotel_index]).sort_values(ascending = False)

    # Get the indexes of the 5 most similar hotels except itself
    top_indexes = list(score_series.iloc[1:6].index)
    
    # Populate the list with the names of the top 5 matching hotels
    for i in top_indexes:
        recommended_hotels.append(list(df.index)[i])
        
    return recommended_hotels

### Make hotel recommendations

In [26]:
recommendations('Motel 6 Seattle Sea-Tac Airport South')

['Emerald Motel',
 'Ramada by Wyndham SeaTac Airport',
 'DoubleTree by Hilton Hotel Seattle Airport',
 'Red Roof Inn Seattle Airport - SEATAC',
 'Country Inn & Suites by Radisson, Seattle-Tacoma International Airport']

In [27]:
recommendations("The Bacon Mansion Bed and Breakfast")

['11th Avenue Inn Bed and Breakfast',
 'Shafer Baillie Mansion Bed & Breakfast',
 'Gaslight Inn',
 'Chittenden House Bed and Breakfast',
 'Bed and Breakfast Inn Seattle']

In [28]:
recommendations("Holiday Inn Seattle Downtown")

['Holiday Inn Express & Suites Seattle-City Center',
 'Silver Cloud Hotel - Seattle Stadium',
 'Inn at Queen Anne',
 'Renaissance Seattle Hotel',
 'Best Western Plus Pioneer Square Hotel Downtown']